In [1]:
#Recall that a "basis" is the fundamental building block of a system's state
#the state of a system is a linear combination of these basis state. These basis states have coefficients attached to them. For example, 0.6|00> + 0.4|11> means there's a 60% chance of being in the |00> state and 40% change of being in the |11> state

#How to represent this in python?
# We need to store two things for each basis state that is part of our system's state - the basis itself and the coefficient attached to it. 
#We can do that with a dictionary in Python
# The basis state can be the "key" and the coefficient can be the value

#We need to create a class called CorrelationGame for this project
#When we create an instance of the class, it should start with a single bit in state |0>. This means that the initial bit is 1.0|0>. {"0": 1.0}
#We also need to keep track of the number of bits in the system during this time


In [8]:
import random
import numpy as np

class CorrelationGame():
    def __init__(self):
        self.state = {"0": 1.0}  # We are using "self" so that other methods inside the class can access these variables.
        self.num_bits = 1  
        # Initialize two sets to keep track of correlated/uncorrelated bits. These will store the indices of the bits.
        self.uncorrelated_bits = {0}
        self.correlated_bits = set()
        
    # Create a method to add a new classical bit to a specified state. The state is picked randomly if the state value is -1
    def add_a_new_bit(self, state_value=-1):  # Default state value is -1
        current_bit_length = self.num_bits

        if state_value == -1:
            state_value = random.randint(0, 1)  # Choose ONE random bit for all states

        if state_value == 1:
            new_state = {key + "1": value for key, value in self.state.items()}  # Using dictionary comprehension to create a new dictionary by modifying the keys of the existing dictionary
        elif state_value == 0:
            new_state = {key + "0": value for key, value in self.state.items()}
        else:
            raise ValueError("Invalid state_value. Must be 0, 1, or -1.")

        self.state = new_state
        self.num_bits += 1  
        self.uncorrelated_bits.add(current_bit_length)

    # Check the system's current state
    def print_state(self):
        print(self.state)
        
    # Print the coefficients
    def print_state_vector(self):
        state_vector = list(self.state.values())  # Convert the dictionary values into a list 
        print(state_vector)
    
    # NOT operation    
    def NOT(self, index_of_bit):
        new_state = {}  # Create a new dictionary to store the updated states

        for key, value in self.state.items():  # For each key-value pair
            key_list = list(key)  # Key is a string, and strings are immutable in Python, so we need to create a list to allow modifications in strings
            if index_of_bit < len(key_list):  # Ensure that the index is valid   
                # Flip the specified bit
                key_list[index_of_bit] = "1" if key_list[index_of_bit] == "0" else "0"
            # Convert the list back to a string
            new_key = "".join(key_list)
            new_state[new_key] = value
        # Update the state of the system
        self.state = new_state
  
    # CNOT operation
    def CNOT(self, control_bit, target_bit):
        # Handle edge cases where the number of bits is less than one or the control bit is equal to the target bit        
        if self.num_bits < 2:
            raise ValueError("CNOT operation requires at least two bits.")
        if control_bit == target_bit:
            raise ValueError("Control bit and target bit cannot be the same.")
        if control_bit >= self.num_bits or target_bit >= self.num_bits:
            raise ValueError("Bit indices out of range.")  # Ensure valid bit indices

        # Since we can't modify the keys in a dictionary directly, we need to create a new dictionary again
        new_state = {}
        for key, value in self.state.items():
            key_list = list(key)

            if key_list[control_bit] == "1":
                key_list[target_bit] = "1" if key_list[target_bit] == "0" else "0"
            # Convert the list back to a string
            new_key = "".join(key_list)
            new_state[new_key] = value
        # Update the state of the system
        self.state = new_state
        
        self.uncorrelated_bits.discard(control_bit)  # Use discard to safely remove if present. Discard does nothing if the element is not present in the set
        self.uncorrelated_bits.discard(target_bit)  # Discard target_bit as well
        self.correlated_bits.add(control_bit)      # Add control_bit to correlated set
        self.correlated_bits.add(target_bit)       # Add target_bit too

    def randomCNOT(self):
        if self.num_bits < 2:
            raise ValueError("CNOT operation requires at least two bits.")
            
        # We can use the sample() method to generate two DISTINCT random bits from the range of available bits
        control_bit, target_bit = random.sample(range(self.num_bits), 2) 
        # Apply the CNOT operation
        self.CNOT(control_bit, target_bit)
        
    # Correlation in this context means the state of one bit depends on the state of another bit. 
    # For example, if you have a state like 0.5|00> + 0.5|11>, bits 0 and 1 are correlated. 
    # If bit 0 is 0, then bit 1 is also 0. If bit 0 is 1, then bit 1 is also 1. They are always the same.
    
    # NOTE: When we add a new bit to the system, all the bits might be uncorrelated. Performing the CNOT operation introduces correlation 
    # Recall that CNOT operation means that the state of the target_bit is dependent on the control_bit
    # We can keep track of which bits we have applied CNOT operations to. 
    # If a bit has been involved in a CNOT as a control or target, we can consider it "correlated".
    
    # Method to find out if a specified bit is correlated
    def is_correlated(self, index_of_bit):
        return index_of_bit in self.correlated_bits


    # Method to find out if a specified bit is uncorrelated. We need to convert the self.uncorrelated_bits set to a list and then return the uncorrelated bit indices
    def get_uncorrelated_bits(self):
        return list(self.uncorrelated_bits)

    # Similarly, return the indices of correlated bits
    def get_correlated_bits(self):
        return list(self.correlated_bits)

    def create_correlations(self):
        uncorrelated_bit_indices = self.uncorrelated_bits.copy()  # Iterate over the copy to avoid issues if set changes during iteration
        for uncorrelated_bit_index in uncorrelated_bit_indices:
            possible_target_indices = []  # Initialize possible target indices for *each* uncorrelated bit
            for i in range(self.num_bits):
                if i != uncorrelated_bit_index:
                    possible_target_indices.append(i) #a more efficient way is using list comprehension ------ possible_target_indices = [i for i in range(self.num_bits) if i != uncorrelated_bit_index]
            if possible_target_indices:  # Check if there are any possible target indices in the list
                target_bit_index = random.choice(possible_target_indices)  # Choose a random index from the possible target indices
                self.CNOT(uncorrelated_bit_index, target_bit_index)  # Apply CNOT to create correlation  
                
    def remove_an_uncorrelated_bit(self):
        # Handle the case when there's only one or zero bits in the system
        if self.num_bits <= 1:
            return None
             
        uncorrelated_bit_indices = self.uncorrelated_bits.copy()  # Using .copy() is important as we might modify self.uncorrelated_bits in other methods later, and we want to work with a consistent set here.
        if not uncorrelated_bit_indices:
            return None # No uncorrelated bits to remove

        # Randomly choose a bit index to remove
        remove_bit_index = random.choice(list(uncorrelated_bit_indices)) # We convert the set to a list because random.choice works with sequences like lists.

        # Update state
        new_state = {}
        for basis_state, coefficient in self.state.items():
            new_basis_state = basis_state[:remove_bit_index] + basis_state[remove_bit_index+1:] # Construct a new basis state by taking the part of the original basis_state before the removed_bit_index and concatenating it with the part after the removed_bit_index. This effectively removes the bit at removed_bit_index.
            #add to existing coefficient, or create a new entry
            new_state[new_basis_state] = new_state.get(new_basis_state, 0) + coefficient #get(key, default) ensures that if new_basis_state is not already in new_state, it initializes to 0 before adding the coefficient.
        self.state = new_state
        self.num_bits -= 1 # Reduce the number of bits in the system

        # Updated self.uncorrelated_bits and self.correlated_bits sets
        new_uncorrelated_bits = set()
        for bit_index in self.uncorrelated_bits:
            if bit_index < remove_bit_index:
                new_uncorrelated_bits.add(bit_index)
            elif bit_index > remove_bit_index:
                new_uncorrelated_bits.add(bit_index - 1)

        new_correlated_bits = set()
        for bit_index in self.correlated_bits:
            if bit_index < remove_bit_index:
                new_correlated_bits.add(bit_index)
            elif bit_index > remove_bit_index:
                new_correlated_bits.add(bit_index - 1)

        self.correlated_bits = new_correlated_bits
        self.uncorrelated_bits = new_uncorrelated_bits

        return remove_bit_index

    def remove_uncorrelated_bits(self):
        if(self.num_bits <= 1):
            return None
        while(self.num_bits > 1):
            if not self.uncorrelated_bits: # If there are no uncorrelated bits, break the loop
                break
            self.remove_an_uncorrelated_bit()
        
#The goal of the test is to simulate the process of adding bits, correlating them, and reducing the system to a single correlated bit. This ultimately reduces the state space. Remember that initially, we start with uncorrelated bits
# Test the implementation with n=10
n = 10
game = CorrelationGame()

# Step 1: Add 9 more bits (total 10 bits)
for _ in range(n - 1):
    game.add_a_new_bit()

# Step 2: Randomly apply CNOT operations
for _ in range(n):
    game.randomCNOT()

# Step 3: Remove uncorrelated bits
game.remove_uncorrelated_bits()

# Step 4: Print results
game_state = game.state
correlated_bits = game.get_correlated_bits()

game_state, correlated_bits


({'11111010': 1.0}, [0, 1, 2, 3, 4, 5, 6, 7])